In [1]:
from itertools import islice
import logging
import os

import pandas as pd
import papermage
from papermage.recipes import CoreRecipe
from papermage.visualizers import plot_entities_on_page
from tqdm.auto import tqdm

/Users/sireeshgururaja/miniconda3/envs/papermage/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
recipe = CoreRecipe()

INFO:CoreRecipe:Instantiating recipe...
INFO:iopath.common.file_io:URL https://www.dropbox.com/s/ukbw5s673633hsw/publaynet-tf_efficientdet_d0.pth.tar?dl=1 cached in /Users/sireeshgururaja/.torch/iopath_cache/s/ukbw5s673633hsw/publaynet-tf_efficientdet_d0.pth.tar?dl=1
INFO:timm.models._helpers:Loaded state_dict from checkpoint '/Users/sireeshgururaja/.torch/iopath_cache/s/ukbw5s673633hsw/publaynet-tf_efficientdet_d0.pth.tar?dl=1'
INFO:CoreRecipe:Finished instantiating recipe


In [10]:
BASE_FOLDER = "/Users/sireeshgururaja/datasets/DoD SAFE-sYt9ADYMsTiZ5uVB/even-years"
OUTPUT_PATH = "/Users/sireeshgururaja/datasets/DoD_SAFE_Papermage"
N_PAPERS_PER_YEAR = 5

papers_dict = []
page1_images = {}

for year in tqdm(os.listdir(BASE_FOLDER)):
    if not year.isnumeric():
        continue
    for paper_filename in islice(os.listdir(os.path.join(BASE_FOLDER, year)), N_PAPERS_PER_YEAR):
        if not paper_filename.lower().endswith(".pdf"):
            continue
        abs_filename = os.path.join(BASE_FOLDER, year, paper_filename)
        doc = recipe.from_pdf(abs_filename)
        n_sentences = len(doc.sentences)
        doc_sections = ", ".join([section.text for section in doc.sections])
        try:
            abstract = doc.abstracts[0].text
            title = doc.titles[0].text
            authors = doc.authors[0].text
        except:
            abstract = None
            title = None
            authors = None

        papers_dict.append({
            "year": year,
            "filename": paper_filename,
            "n_sentences": n_sentences,
            "title": title,
            "authors": authors,
            "abstract": abstract,
            "sections": doc_sections
        })

        page = doc.pages[0]
        highlighted = plot_entities_on_page(page.images[0], page.tokens, box_width=0, box_alpha=0.3, box_color="yellow")
        highlighted = plot_entities_on_page(highlighted, page.abstracts, box_width=2, box_alpha=0.1, box_color="red")
        highlighted = plot_entities_on_page(highlighted, page.sections, box_width=1, box_alpha=0.1, box_color="green")

        highlighted.save(os.path.join(OUTPUT_PATH, f"{year}_{paper_filename}".lower().replace(".pdf", ".png")), is_overwrite=True)

papers_df = pd.DataFrame(papers_dict)


  0%|          | 0/20 [00:00<?, ?it/s]INFO:CoreRecipe:Parsing document...
INFO:CoreRecipe:Rasterizing document...
INFO:CoreRecipe:Predicting words...
INFO:CoreRecipe:Predicting sentences...
INFO:CoreRecipe:Predicting blocks...
100%|██████████| 6/6 [00:06<00:00,  1.02s/it]
INFO:CoreRecipe:Predicting vila...
INFO:CoreRecipe:Parsing document...
INFO:CoreRecipe:Rasterizing document...
INFO:CoreRecipe:Predicting words...
INFO:CoreRecipe:Predicting sentences...
INFO:CoreRecipe:Predicting blocks...
100%|██████████| 16/16 [00:16<00:00,  1.01s/it]
INFO:CoreRecipe:Predicting vila...
INFO:CoreRecipe:Parsing document...
INFO:CoreRecipe:Rasterizing document...
INFO:CoreRecipe:Predicting words...
INFO:CoreRecipe:Predicting sentences...
INFO:CoreRecipe:Predicting blocks...
100%|██████████| 13/13 [00:13<00:00,  1.01s/it]
INFO:CoreRecipe:Predicting vila...
INFO:CoreRecipe:Parsing document...
INFO:CoreRecipe:Rasterizing document...
INFO:CoreRecipe:Predicting words...
INFO:CoreRecipe:Predicting sentences

ValueError: Failed to attach. 90 `images` != 89 pages in `doc`.

In [11]:
papers_df

NameError: name 'papers_df' is not defined